In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
load_dotenv()

api_key = os.getenv("OPEN_AI_KEY")
organization = os.getenv("OPEN_AI_ORG")

from langchain.llms import OpenAI
llm = OpenAI(
    temperature=0,
    openai_api_key=api_key,
    openai_organization=organization,
)

# Tagging
Tagging has a few components:

function: Like extraction, tagging uses functions to specify how the model should tag a document
schema: defines how we want to tag the document

In [4]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.chat_models import ChatOpenAI

# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

# LLM
llm = ChatOpenAI(
    temperature=0,
    openai_api_key=api_key,
    openai_organization=organization,
)
chain = create_tagging_chain(schema, llm)
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

{'sentiment': 'positive', 'language': 'Spanish'}

## Finer control
Careful schema definition gives us more control over the model’s output.

Specifically, we can define:

possible values for each property
description to make sure that the model understands the property
required properties to be returned
Here is an example of how we can use _enum_, _description_, and _required_ to control for each of the previously mentioned aspects:



In [ ]:
schema = {
    "properties": {
        "aggressiveness": {
            "type": "integer",
            "enum": [1, 2, 3, 4, 5],
            "description": "describes how aggressive the statement is, the higher the number the more aggressive",
        },
        "language": {
            "type": "string",
            "enum": ["spanish", "english", "french", "german", "italian"],
        },
    },
    "required": ["language", "sentiment", "aggressiveness"],
}

In [ ]:
chain = create_tagging_chain(schema, llm)

In [6]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

{'sentiment': 'positive', 'language': 'Spanish'}

In [7]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.run(inp)

{'sentiment': 'enojado', 'aggressiveness': 1, 'language': 'es'}

In [8]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.run(inp)

{}

## Pydantic
We can also use a Pydantic schema to specify the required properties and types.

We can also send other arguments, such as enum or description, to each field.

This lets us specify our schema in the same manner that we would a new class or function in Python with purely Pythonic types.

In [9]:
from langchain.pydantic_v1 import BaseModel, Field

class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

chain = create_tagging_chain_pydantic(Tags, llm)

In [11]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = chain.run(inp)
res

Tags(sentiment='sad', aggressiveness=5, language='spanish')

# Entity
Entity memory remembers given facts about specific entities in a conversation. 
It extracts information on entities (using an LLM) and builds up its knowledge about that entity over time.

In [7]:
from langchain.memory import ConversationEntityMemory

memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Chaky & Gun are working on a NLP course"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": "That sounds like a great project! What kind of project are they working on?"}
)

memory.load_memory_variables({"input": 'who is Chaky'})

{'history': 'Human: Chaky & Gun are working on a NLP course\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Chaky': 'Chaky is working on a NLP course with Gun.',
  'Gun': 'Gun is working on a NLP course.'}}

In [10]:
memory = ConversationEntityMemory(llm=llm, return_messages=True)
_input = {"input": "Chaky & Gun are working on a NLP course"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

memory.load_memory_variables({"input": 'who is Chaky'})

{'history': [HumanMessage(content='Chaky & Gun are working on a NLP course'),
  AIMessage(content=' That sounds like a great project! What kind of project are they working on?')],
 'entities': {'Chaky': 'Chaky is working on a NLP course with Gun.',
  'Gun': 'Gun is working on a NLP course.'}}

#### Using in chain

In [11]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

conversation.predict(input="Chaky & Gun are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like a great project! What kind of project are they working on?'